In [4]:
import pandas as pd
import numpy as np
import joblib
import pickle
from datetime import datetime, timedelta
from google.colab import drive

# === Mount Drive ===
drive.mount('/content/drive')

class ZoneForecastSystem:
    def __init__(self, kmeans_path, scaler_path, cluster_csv_path, arima_model_path):
        self.kmeans = joblib.load(kmeans_path)
        self.scaler = joblib.load(scaler_path)
        self.cluster_summary = pd.read_csv(cluster_csv_path).groupby('cluster')[['average_latency', 'total_throughput']].mean()
        with open(arima_model_path, 'rb') as f:
            self.fitted_arima = pickle.load(f)['fitted_model']

    def label_zone(self, cluster_id):
        throughput = self.cluster_summary.loc[cluster_id, 'total_throughput']
        latency = self.cluster_summary.loc[cluster_id, 'average_latency']
        avg_throughput = self.cluster_summary['total_throughput'].mean()
        avg_latency = self.cluster_summary['average_latency'].mean()
        if throughput > avg_throughput and latency < avg_latency:
            return 'High Performance'
        elif throughput < avg_throughput and latency > avg_latency:
            return 'Low Performance'
        else:
            return 'Moderate'

    def predict_zone(self, avg_latency, bandwidth, throughput, latitude, longitude):
        input_data = pd.DataFrame([[avg_latency, bandwidth, throughput, latitude, longitude]],
                                  columns=['average_latency', 'total_bandwidth', 'total_throughput', 'latitude', 'longitude'])
        scaled_input = self.scaler.transform(input_data)
        cluster = self.kmeans.predict(scaled_input)[0]
        zone = self.label_zone(cluster)
        return cluster, zone

    def forecast_throughput(self, steps, start_time=None):
        if start_time is None:
            start_time = datetime.now()
        future_index = pd.date_range(start=start_time, periods=steps, freq='h')
        exog = pd.DataFrame({
            'hour': future_index.hour,
            'day_of_week': future_index.dayofweek
        }, index=future_index)
        forecast = self.fitted_arima.forecast(steps=steps, exog=exog)
        return forecast, future_index

# === Initialize System ===
system = ZoneForecastSystem(
    "/content/drive/MyDrive/Colab Notebooks/Group Assignment/Version 3/SavedModel/Clustering/kmeans_model.pkl",
    "/content/drive/MyDrive/Colab Notebooks/Group Assignment/Version 3/SavedModel/Clustering/scaler.pkl",
    "/content/drive/MyDrive/Colab Notebooks/Group Assignment/Version 3/SavedModel/Clustering/clustered_50k_output.csv",
    "/content/drive/MyDrive/Colab Notebooks/Group Assignment/Version 3/SavedModel/TimeSeries/arima_model.pkl"
)

# === Main UI Loop ===
while True:
    print("\n==== 5G Network Zone Prediction System ====")
    print("1. Predict Performance Zone (KMeans)")
    print("2. Forecast Throughput (ARIMA)")
    print("0. Exit")

    option = input("Choose an option (0, 1, 2): ").strip()

    if option == "1":
        try:
            print("\n--- Predict Zone ---")
            lat = float(input("Enter Latitude: "))
            lon = float(input("Enter Longitude: "))
            latency = float(input("Enter Average Latency (ms): "))
            bandwidth = float(input("Enter Total Bandwidth (Mbps): "))
            throughput = float(input("Enter Total Throughput (Mbps): "))

            cluster, zone = system.predict_zone(latency, bandwidth, throughput, lat, lon)

            print("\n--- Result ---")
            print(f"Input Location: Latitude={lat}, Longitude={lon}")
            print(f"Network Metrics: Latency={latency} ms, Bandwidth={bandwidth} Mbps, Throughput={throughput} Mbps")
            print(f"Predicted Cluster ID: {cluster}")
            print(f"Performance Classification: {zone}")

        except ValueError:
            print("Invalid input. Please enter numeric values only.")

    elif option == "2":
        print("\n--- Forecast Throughput ---")
        try:


            custom_start = input("Optional: Enter custom forecast start time (YYYY-MM-DD HH:MM) or leave blank: ").strip()
            if custom_start:
                try:
                    start_time = pd.to_datetime(custom_start)
                except Exception:
                    print("Invalid datetime format. Using current time instead.")
                    start_time = None
            else:
                start_time = None

            steps = int(input("Enter number of hours to forecast: "))
            if steps <= 0:
                print("Please enter a positive number.")
                continue

            print(f"\nYou chose to forecast the next {steps} hour(s) of throughput.")

            forecast, times = system.forecast_throughput(steps, start_time)
            print(f"\n--- Forecasted Throughput for {steps} Hour(s) ---")
            for i in range(len(forecast)):
                print(f"{times[i].strftime('%Y-%m-%d %H:%M')}: {forecast.iloc[i]:.2f} Mbps")

            save_csv = input("\nSave forecast to CSV? (y/n): ").lower()
            if save_csv == 'y':
                forecast_df = pd.DataFrame({'timestamp': times, 'forecast_throughput': forecast})
                forecast_df.to_csv("forecast_output.csv", index=False)
                print("Forecast saved to forecast_output.csv")

        except ValueError:
            print("Invalid input. Please enter a valid number.")

    elif option == "0":
        confirm = input("Are you sure you want to exit? (y/n): ").strip().lower()
        if confirm == 'y':
            print("Exiting the system. Goodbye!")
            break
        else:
            continue

    else:
        print("Invalid option. Please enter 0, 1, or 2.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

==== 5G Network Zone Prediction System ====
1. Predict Performance Zone (KMeans)
2. Forecast Throughput (ARIMA)
0. Exit
Choose an option (0, 1, 2): 2

--- Forecast Throughput ---
Optional: Enter custom forecast start time (YYYY-MM-DD HH:MM) or leave blank: 2025/1//23 10:00
Invalid datetime format. Using current time instead.
Enter number of hours to forecast: 2

You chose to forecast the next 2 hour(s) of throughput.

--- Forecasted Throughput for 2 Hour(s) ---
2025-05-26 05:55: 43.52 Mbps
2025-05-26 06:55: 43.16 Mbps

Save forecast to CSV? (y/n): n

==== 5G Network Zone Prediction System ====
1. Predict Performance Zone (KMeans)
2. Forecast Throughput (ARIMA)
0. Exit
Choose an option (0, 1, 2): 2

--- Forecast Throughput ---
Optional: Enter custom forecast start time (YYYY-MM-DD HH:MM) or leave blank: 2025/11/23 10:00
Enter number of hours to forecast: 4

Y

KeyboardInterrupt: Interrupted by user